In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

import collections
def bfs(C, F, l, src, snk) :
  V = len(C)
  Q = collections.deque()
  Q.append(src)
  l[src] = 0
  while Q :
    u = Q.popleft()
    for v in range(V) :
      if l[v] == -1 and C[u][v] - F[u][v] > 0 :
        l[v] = l[u] + 1
        Q.append(v)
  return l[snk] != -1

def dfs(C, F, W, l, u, snk, cur = 1 << 63) :
  if u == snk : return cur
  V = len(C)
  while W[u] < V :
    v = W[u]
    if l[v] == l[u] + 1 and C[u][v] > F[u][v] :
      A = min(cur, C[u][v] - F[u][v])
      tmp = dfs(C, F, W, l, v, snk, A)
      if tmp > 0 :
        F[u][v] += tmp
        F[v][u] -= tmp
        return tmp
    W[u] += 1
  return 0

def dinic(C, src, snk) :
  V = len(C)
  F = [[0] * V for _ in range(V)]
  ret = 0

  while True :
    l = [-1] * V
    W = [0] * V
    if not bfs(C, F, l, src, snk) : break
    while True:
      tmp = dfs(C, F, W, l, src, snk)
      if tmp == 0 : break
      ret += tmp
  return ret, F

def validate(F, L):
  for i, l in enumerate(F) :
    sum = 0
    for v in l :
      sum += v
    if sum != L[i] : 
      return False
  return True

def sol():
  N = int(input())
  L1 = [*map(int, input().split())]
  L2 = [*map(int, input().split())]
  
  V = N * 2 + 2
  src = V - 2
  snk = V - 1
  C = [[0] * V for _ in range(V)]
  for i in range(N) :
    C[src][i] = L1[i]
    C[N+i][snk] = L2[i]

  def desc(c):
    for i in range(N) :
      for j in range(N) :
        C[i][N+j] = c
    
    res, F = dinic(C, src, snk)
    res = [[0] * N for _ in range(N)]

    for i in range(N) : 
      for j in range(N) :
        if not F[i][N+j]: continue
        res[i][j] = F[i][N+j]
    
    return (validate(res, L1) and validate(zip(*res), L2)), res
  
  lo = 0
  hi = ans = 10000
  #이분탐색으로 플로우를 log(N)번 반복해서 최소값을 찾는다.
  while lo + 1 < hi: 
    mid = (lo + hi) // 2
    valid, res = desc(mid)
    if valid:
      ans = mid
      ans2 = res
      hi = mid
    else:
      lo = mid

  sys.stdout.write(str(ans) + '\n')
  for l in ans2 :
    sys.stdout.write(' '.join(map(str, l)) + '\n')

sol()

### 풀이
- 1960(행렬만들기)의 심화문제. source와 sink의 연결까진 똑같으나, 각 정점끼리의 용량이 0혹은 1에서 0~10000으로 확장되었다.
- 이분탐색으로 디닉을 최대 $log_2(10000) = 14$ 번 돌리면서 가능한 용량의 최솟값의 최댓값을 찾는다.